# Visualisation Tools

## Introduction

While numerical metrics provide a quantitative measure of uncertainty, visualizations offer an intuitive, qualitative understanding of a model's predictions. A single plot can often reveal the nature of a model's confidence in a way that is difficult to grasp from numbers alone. To this end, probly provides specialized tools for plotting predictive distributions, focusing on classification tasks.

Currently, the plotting module is designed to visualize the outputs of 3-class classifiers on a 2D simplex. This is a triangle where each corner represents 100% confidence in one of the three classes.

The library provides two main functions for this:

    The Simplex Plot is used to scatter individual predictions on this triangle, allowing you to see the distribution of a model's predictions across a dataset.

    The Credal Set Plot is a more advanced tool for visualizing the uncertainty of a single prediction derived from multiple samples (e.g., from an ensemble or MC-Dropout). It plots the region of all possible probability distributions consistent with the model's outputs. A small, tight region indicates high confidence, while a large, spread-out region signifies high uncertainty.

These tools are essential for the qualitative interpretation and debugging of classification uncertainty.

## 0. First things first:
All modules mentioned here have more in depth notebooks to explain more detailed, how they are working.
You can find them under the section further reading.

## 1. What can you visualize in probly? 
### Credal Sets: <br>
Based on the number of classes given in the visualizer, the `credal` package chooses between those three modules:

- **Intervall plot** / `plot_2d`: Quick sanity checks for two-feature views (or 2D embeddings): cluster overlap, separation, and how uncertainty behaves in feature space.

- **Ternary plot** / `plot_3d`: Visualization of **3-class probability vectors** on a simplex: highlights confident regions (near corners) vs. ambiguous regions (towards the center / edges).

- **Spider plot** / `plot_multid`: Multi-class probability diagnostics when `n_classes > 3`: compact views to spot systematic uncertainty patterns across many classes.


### Specialized plots:<br>
Upon our credal package, you can also visualize special plots:
- `plot_uncertainty`: Visualizes **margin-based confidende** between two 2D clusters by mapping the SVM decision margin into a smooth field.

- `plot_dirichlet`: Plots **Dirichlet density contours** on the ternary simplex to illustrate “distributions over probabilities” (concentration, symmetry, and skew).

- `plot_coverage_efficency`: Shows the **trade-off between empirical coverage and prediction set size** across confidence levels (coverage vs. efficiency on a dual-axis view)

## 2. Imports
To demonstrate, first let's import the following modules:

In [ ]:
# Import modules needed
print("Imports successfully loaded.")

## 3. Credal
What the package does:
It takes an array of probability vectors (each summing to 1) and visualizes them as a credal set:
- **2 classes:** interval band on a line
- **3 classes:** ternary simpplotlex with convex hull
- **more than 3** classes: radar/spider plot with min–max envelope
The public entry point is `create_credal_plot()`, which delegates to `dispatch_plot()` based on the number of classes.

### 3.1 Public API `credal_visualization.py`
This module provides the single public entry point for creating credal-set visualizations across different class counts (2D / 3D / multi-class).
`create_credal_plot()` takes the `input_data` plus optional flags to customize your plot. 
It delegates all validation and plot selection to `dispatch_plot()` from `input_handling.py`, which decides which plot backend to use based on the number of classes.
If `show=True`, it calls `plt.show()` so the figure is displayed immediately.

In [ ]:
#load credal data for all three plots here.
print("Demodata successfully loaded.")

### 3.2 `input_handling.py`
It selects and executes the correct plotting function based on class count.
By calling `create_credal_plot()` from `credal_visualization.py` your 

Validates input_data is a non-empty NumPy array, at least 2D, with non-negative probabilities that sum to 1 per row, and ≥2 classes with `check_shape()`.
It then normalizes shapes by flattening higher-dimensional inputsso plotting always sees a 2D matrix of samples with `normalize_input()`.
**Important to mention:**<br>
- Your probabilities **must sum to 1**(otherwise you’ll get a clear error).
- `choice="Probability"` shows only raw points; `choice="MLE"` shows only the mean; `choice="Credal"` shows only the uncertainty region; `choice=None` shows both.
- `minmax=True` only has an effect for **3-class plots**.

```python
